# Métodos numéricos para resolver el problema de caida libre

Un paracaidista con una masa de 87.5 kg salta de un globo aerostático fijo.
Tomando en cuenta la expresión de un modelo matemático simple, calcular la
velocidad terminal del paracaidista. Considere que el coeficiente de arrastre es
igual a 13.5 kg/s.

Se presentan los siguientes métodos:
* Euler
* Euler modificado
* Runge-Kutta de orden 4

#### La siguiente celda solo se debe ejecutar la primera vez que se use el notebook para instalar las bibliotecas necesarias.

In [ ]:
# Importar las bibliotecas
from sympy import lambdify, sympify # ¡¡¡NO MODIFICAR!!!
from sympy.abc import t, v # ¡¡¡NO MODIFICAR!!!
import plotly.express as px # ¡¡¡NO MODIFICAR!!!
import plotly.graph_objects as go # ¡¡¡NO MODIFICAR!!!
import numpy as np # ¡¡¡NO MODIFICAR!!!
import metodos_numericos_dcb_fi.utilidades as ut

### Definición de constantes y funciones

In [ ]:
t_0 = 0 # Tiempo inicial en minutos
t_f = 25 # Tiempo final en minutos
valores_t = np.linspace(t_0, t_f, (t_f-t_0)*15) # Vector de tiempo !!!NO MODIFICAR!!!
gravedad = 9.81 # gravedad en m/s^2

masa = 87.5 # masa en kg
k = 13.5  # coeficiente de arrastre en kg/s
C = -masa*gravedad/k # Constante de integración

# Ecuacion diferencial de la forma dy/dx = f(x, y)
f_real = ut.convertirFuncion(f'{gravedad}-{k/masa}*v', 'v') # No se coloca explicitamente la variable independiente y la derivada
f_sol_analitica = ut.convertirFuncion(f'{masa*gravedad/k}+{C}*exp(-{k/masa}*t)', 't') # Solucion analitica de la ecuacion diferencial ¡¡¡NO MODIFICAR!!!

### Definición de constante para métodos numéricos

In [ ]:
h = 2 # Tamaño de paso en minutos
n_pasos = int(np.ceil((t_f-t_0)/h)) # Número de pasos !!!NO MODIFICAR!!!
y_0 = f_sol_analitica.f(t_0) # Velocidad inicial !!!NO MODIFICAR!!!

### Analitica

In [ ]:
def solucion_analitica(f_, valores_t, C, k):
    f = f_.f # Obtener la funcion evaluable
    valores_y = list(map(lambda t: f(t), valores_t)) # Evaluar la función en el vector de tiempo

    # Graficar
    titulo = f'Caida libre<br>Solucion analítica: v(t)={round(masa*gravedad/k,3)}{+round(C,3)}*exp(-{round(k/masa,3)}*t)' # Titulo de la gráfica
    eje_x = 't[s]' # Nombre del eje x
    eje_y = 'v(t)[m/s]' # Nombre del eje y 

    return ut.graficar(valores=[valores_t, valores_y], titulo=titulo, labels=[eje_x, eje_y], metodo='Analitica') # Regresar gráfica de la solución analítica

plot_Analitica = solucion_analitica(f_sol_analitica, valores_t, C, k) # Graficar solución analítica

### Euler

In [ ]:
def metodo_euler(f_, t_0, y_0, h, n_pasos):
    f = f_.f # Obtener la función evaluable
    valores_y_Euler = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo
    
    # Iterar sobre el número de pasos y aplicar el método de Euler
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h) # Agregar el valor de t a la lista
        y_nueva = valores_y_Euler[i] + h * f(valores_y_Euler[i]) # Método de Euler
        valores_y_Euler.append(y_nueva) # Agregar el valor de y a la lista

    # Graficar
    titulo = f'Caida libre<br>Método de Euler con h = {round(h,3)}, n = {n_pasos} y T(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[s]' # Nombre del eje x
    eje_y = 'v(t)[m/s]' # Nombre del eje y

    return ut.graficar(valores=[valores_t, valores_y_Euler], titulo=titulo, labels=[eje_x, eje_y], metodo='Euler') # ¡¡¡NO MODIFICAR!!!
 

metodo_euler(f_real, t_0, 0, 0.5, 50) # ¡¡¡NO MODIFICAR!!!

plot_Euler = metodo_euler(f_real, t_0, y_0, h, n_pasos) # Graficar método de Euler

### Euler modificado

In [ ]:
def metodo_euler_mod(f_, t_0, y_0, h, n_pasos):
    f = f_.f # Obtener la función evaluable
    valores_y_Euler_Mod = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo

    # Iterar sobre el número de pasos y aplicar el método de Euler modificado
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h)
        y_nueva = valores_y_Euler_Mod[i] + h * f(valores_y_Euler_Mod[i] + h/2 * f(valores_y_Euler_Mod[i])) # ¡¡¡NO MODIFICAR!!!
        valores_y_Euler_Mod.append(y_nueva) # ¡¡¡NO MODIFICAR!!!

    # Graficar
    titulo = f'Caida libre<br>Método de Euler modificado con h = {round(h,3)}, n = {n_pasos} y N(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[s]' # Nombre del eje x
    eje_y = 'v(t)[m/s]' # Nombre del eje y

    return ut.graficar(valores=[valores_t, valores_y_Euler_Mod], titulo=titulo, labels=[eje_x, eje_y], metodo='Euler M') # ¡¡¡NO MODIFICAR!!!

plot_Euler_Mod = metodo_euler_mod(f_real, t_0, y_0, h, n_pasos) # Graficar método de Euler modificado

## Runge-Kutta de orden 4

In [ ]:
def metodo_RK_4(f_, t_0, y_0, h, n_pasos):
    f = f_.f # Obtener la función evaluable
    valores_y_RK_4 = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo

    # Iterar sobre el número de pasos y aplicar el método de Runge-Kutta 4
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h) # ¡¡¡NO MODIFICAR!!!
        k1 = h * f(valores_y_RK_4[i]) # ¡¡¡NO MODIFICAR!!!
        k2 = h * f(valores_y_RK_4[i] + k1/2) # ¡¡¡NO MODIFICAR!!!
        k3 = h * f(valores_y_RK_4[i] + k2/2) # ¡¡¡NO MODIFICAR!!!
        k4 = h * f(valores_y_RK_4[i] + k3) # ¡¡¡NO MODIFICAR!!!
        y_nueva = valores_y_RK_4[i] + 1/6 * (k1 + 2*k2 + 2*k3 + k4) # ¡¡¡NO MODIFICAR!!!
        valores_y_RK_4.append(y_nueva) # ¡¡¡NO MODIFICAR!!!

    # Graficar
    titulo = f'Caida libre<br>Método de Runge-Kutta 4 con h = {round(h,3)}, n = {n_pasos} y N(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[s]' # Nombre del eje x
    eje_y = 'v(t)[m/s]' # Nombre del eje y

    return ut.graficar(valores=[valores_t, valores_y_RK_4], titulo=titulo, labels=[eje_x, eje_y], metodo='RK4') # ¡¡¡NO MODIFICAR!!!

plot_RK_4 = metodo_RK_4(f_real, t_0, y_0, h, n_pasos) # Graficar método de Runge-Kutta 4

## Comparacion de los metodos

In [ ]:
titulo = f'Caida libre<br>Comparación de metodos numericos y solucion analitica' # ¡¡¡NO MODIFICAR!!!
eje_x = 't[s]' # Nombre del eje x
eje_y = 'v(t)[m/s]' # Nombre del eje y

ut.compararMetodos()